# **Vivli AMR Challenge - Data Processing**


Install Required Packages (Uncomment if needed)

In [ ]:
!pip install econml scikit-learn pandas matplotlib seaborn dowhy

Import Libraries

In [ ]:
from google.colab import drive
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from econml.dml import CausalForestDML
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from dowhy import CausalModel
from econml.dml import LinearDML, CausalForestDML,NonParamDML,DML
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.linear_model import LassoCV
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from econml.score import RScorer

Mount Drive

In [ ]:
drive.mount('/content/drive/') # Mounting Drive
os.chdir('/content/drive/My Drive/AMR Challenge') # Changing Directory

Read Data

In [ ]:
df = pd.read_excel("2025_03_11 atlas_antibiotics.xlsx")
df.head(10)

*   Consider Meropenem Resistance
*   Consider Enterobacterales



In [ ]:
df2 = df[['Species','Family','Country','Gender','Age Group','Speciality','Source','In / Out Patient','Year','Meropenem','Meropenem_I']]

In [ ]:
df_ent = df2[df2['Family'] == 'Enterobacteriaceae']
df_ent.head(5)

df_ent["MEM_resistant"] = df_ent['Meropenem_I'].apply(
    lambda x: 1 if pd.notna(x) and x == 'Resistant' else 0
)

# Encode Categorical Variables
# Define covariates
categorical_cols = ['Source', 'Age Group','Speciality', 'In / Out Patient']

# One-hot encode
df_ent = pd.get_dummies(df_ent, columns=categorical_cols, drop_first=True)
df_ent = df_ent.drop(columns=['Family','Meropenem','Meropenem_I'],axis =1 )



Policy was enacted in 2015 in Greece

In [ ]:
df_gr = df_ent[df_ent['Country'] == 'Greece'].drop(['Country','Species'],axis =1)

binary_cols = df_gr.columns.difference(["Year", "MEM_resistant",'Gender'], sort=False)
df_sum =(df_gr.groupby(["Year"])
      .agg(
          MEM_resistance_rate=("MEM_resistant", "mean"),
        **{col: (col, "sum") for col in binary_cols},
        male_ratio=("Gender", lambda x: np.mean(x == "Male"))
      )
      .reset_index()
)

df_sum['policy'] = (df_sum['Year'] >= 2015).astype(int)

In [ ]:
Y = df_sum['MEM_resistance_rate']
T = df_sum['policy']
X = df_sum[[col for col in df_sum.columns if col not in ['MEM_resistance_rate', 'policy', 'Year']]]

Model Development

In [ ]:
model_configs_outcome = [
    {
        'name': 'RandomForest',
        'estimator': RandomForestRegressor(random_state=0),
        'param_grid': {
            'n_estimators':    (10, 30, 50, 100, 200),
            'max_depth':       [3, 5, None]
          #  'model__min_samples_leaf':[1, 5, 10]
        }
    },
    {
        'name': 'GradientBoosting',
        'estimator': GradientBoostingRegressor(random_state=0),
        'param_grid': {
            'n_estimators':    (50, 100, 200),
            'max_depth':       [3, 5,None],
            'learning_rate':   [0.1, 0.05]
        }
    },
    {
        'name': 'LassoCV',
        'estimator': LassoCV(cv=5, random_state=0),
        'param_grid': {}  # LassoCV does its own CV over alphas
    }
]

model_configs_treatment = [
    {
        'name': 'RandomForest',
        'estimator': RandomForestClassifier(random_state=0),
        'param_grid': {
            'n_estimators':    (10, 30, 50, 100, 200),
            'max_depth':       [3, 5, None]
          #  'model__min_samples_leaf':[1, 5, 10]
        }
    },
    {
        'name': 'GradientBoosting',
        'estimator': GradientBoostingClassifier(random_state=0),
        'param_grid': {
            'n_estimators':    (50, 100, 200),
            'max_depth':       [3, 5,None],
            'learning_rate':   [0.1, 0.05]
        }
    },
    {
        'name': 'LassoCV',
        'estimator': LassoCV(cv=5, random_state=0),
        'param_grid': {}  # LassoCV does its own CV over alphas
    }
]

# Loop, fit GridSearchCV, collect results
results_treatment = []
for cfg in model_configs_outcome:
    gs = GridSearchCV(
        estimator=cfg['estimator'],
        param_grid=cfg['param_grid'],
        scoring='neg_mean_squared_error',
        cv=10,
        n_jobs=-1
    )
    print(f"→ Tuning {cfg['name']}...")
    gs.fit(X, T)
    results_treatment.append({
        'name':         cfg['name'],
        'best_score':   gs.best_score_,
        'best_params':  gs.best_params_,
       'best_estimator': gs.best_estimator_
    })


results_outcome = []
for cfg in model_configs_treatment:
    gs = GridSearchCV(
        estimator=cfg['estimator'],
        param_grid=cfg['param_grid'],
        scoring='neg_mean_squared_error',
        cv=10,
        n_jobs=-1
    )
    print(f"→ Tuning {cfg['name']}...")
    gs.fit(X, Y)
    results_outcome.append({
        'name':         cfg['name'],
        'best_score':   gs.best_score_,
        'best_params':  gs.best_params_,
       'best_estimator': gs.best_estimator_
    })

In [ ]:


#Find & store the best outcome model
best_outcome = max(results_outcome, key=lambda r: r['best_score'])
best_outcome_name   = best_outcome['name']
best_outcome_score  = best_outcome['best_score']
best_outcome_params = best_outcome['best_params']
best_model_y        = best_outcome['best_estimator']

#Find & store the best treatment model
best_treatment = max(results_treatment, key=lambda r: r['best_score'])
best_treatment_name   = best_treatment['name']
best_treatment_score  = best_treatment['best_score']
best_treatment_params = best_treatment['best_params']
best_model_t          = best_treatment['best_estimator']

#Print a summary (or use in f‑strings later)
print(f"🏆 Best Outcome Model  : {best_outcome_name}")
print(f"   • neg‑MSE        = {best_outcome_score:.4f}")
print(f"   • params         = {best_outcome_params}\n")

print(f"🏆 Best Treatment Model: {best_treatment_name}")
print(f"   • score (AUC or neg‑MSE) = {best_treatment_score:.4f}")
print(f"   • params                = {best_treatment_params}\n")



In [ ]:
# split data in train-validation
X_train, X_val, T_train, T_val, Y_train, Y_val = train_test_split(X, T, Y, test_size=.4)

In [ ]:
cf_estimator = CausalForestDML(
    model_t=best_model_t,
    model_y=best_model_y,
    discrete_treatment=True,
    n_estimators=200,
    random_state=0
)


cf_dw = cf_estimator.dowhy.fit(
    Y_train,
    T_train,
    X_train)

Refutation Test

In [ ]:
print(cf_dw.refute_estimate(method_name="random_common_cause"))
print(cf_dw.refute_estimate(method_name="placebo_treatment_refuter", placebo_type="permute"))
print(cf_dw.refute_estimate(method_name="data_subset_refuter"))

Estimate effect

In [ ]:
df_sum['tau_hat'] = cf_dw.effect(X_val)

Plotting

In [ ]:
df_sum['resistance_cf'] = df_sum['MEM_resistance_rate'] + df_sum['policy']*df_sum['tau_hat']
plt.figure(figsize=(8,4))
plt.plot(df_sum['Year'], df_sum['MEM_resistance_rate'], 'o-', label='Observed Resistance')
plt.plot(df_sum['Year'], df_sum['resistance_cf'], 's--', label='Simulated (No Policy)')
plt.xlabel('Year')
plt.ylabel('Resistance Rate (%)')
plt.title('Observed vs Counterfactual Resistance\n(What if Policy Had Not Been Enacted?)')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
df_sum['Y0_hat'] = df_sum['MEM_resistance_rate'] - df_sum['policy'] * df_sum['tau_hat']

# Scenario A: never roll it out
df_sum['policy_never'] = 0
df_sum['resistance_never'] = df_sum['Y0_hat']  # always Y(0)

# Scenario B: roll out in 2012
early_year = 2012
df_sum['policy_2012'] = (df_sum['Year'] >= early_year).astype(int)
df_sum['resistance_2012'] = df_sum['Y0_hat'] + df_sum['policy_2012'] * df_sum['tau_hat']

# Aggregate all 4 curves
agg = df_sum.groupby('Year').agg({
    'MEM_resistance_rate':      'mean',
    'resistance_cf':   'mean',  # = rolled out in 2015?  (same as observed minus orig effect)
    'resistance_never':'mean',
    'resistance_2012': 'mean'
}).reset_index().rename(columns={
    'MEM_resistance_rate':      'Observed',
    'resistance_cf':   'Rollout=2015',
    'resistance_never':'Rollout=None',
    'resistance_2012': 'Rollout=2012'
})

#Plot
plt.figure(figsize=(8,4))
for col, style in [
    ('Observed',      'o-'),
    ('Rollout=None',  's--'),
   # ('Rollout=2015',  'd:'),
    ('Rollout=2012',  'x-.'),
]:
    plt.plot(agg['Year'], agg[col], style, label=col)

plt.xlabel('Year')
plt.ylabel('Resistance (%)')
plt.title('Observed vs. Counterfactual Rollout Scenarios')
plt.axvline(2015, color="blue", linestyle="--", alpha=0.6, label="Policy Enacted 2015")
plt.axvline(2012, color="red", linestyle="--", alpha=0.6, label="Policy Enacted 2012")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.xticks(df_sum['Year'].unique(), rotation=45)
plt.show()